# Imports

In [1]:
import geopandas as gpd
from shapely.wkt import loads
import pandas as pd
from shapely.geometry import Point, MultiPolygon

# Read Data

In [2]:
outlines = gpd.read_file('data/employment_center_outlines.csv').drop(columns=['geometry'])
outlines['the_geom'] = outlines['the_geom'].apply(loads)
outlines = outlines.set_geometry('the_geom')

In [3]:
tracts = gpd.read_file('data/tracts.csv').drop(columns=['geometry'])
tracts['the_geom'] = tracts['the_geom'].apply(loads)
tracts = tracts.set_geometry('the_geom')

In [4]:
income = gpd.read_file('data/income_estimates.csv').drop(columns=['geometry'])

# Merge Tracts and Outlines on Centroids

In [5]:
def get_centroid(df):
    polygons = df['the_geom']
    if isinstance(polygons, MultiPolygon) or isinstance(polygons, Polygon):
        return polygons.centroid
    else:
        return None

In [6]:
outlines['centroid'] = outlines.apply(get_centroid, axis=1)
tracts['centroid'] = tracts.apply(get_centroid, axis=1)
outlines_and_tracts = outlines.sjoin_nearest(tracts, how='left', distance_col='centroid')

# Merge Tracts and Outlines on Income

In [7]:
income['households']=income['households'].astype(int)
maxes = income[income['yr_id']=='2022'].groupby(['tract']).max()[['households']].reset_index()
maxes['tract'] = maxes['tract'].astype(str)
merged_not_equal = income.merge(maxes,how='right',on='tract', suffixes=('_right', '_left'))
merge_income= merged_not_equal[merged_not_equal['households_right']==merged_not_equal['households_left']][merged_not_equal['yr_id']=='2022'][merged_not_equal['households_left'].astype(int)>0]

<ipython-input-7-16e2f209a8e4>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merge_income= merged_not_equal[merged_not_equal['households_right']==merged_not_equal['households_left']][merged_not_equal['yr_id']=='2022'][merged_not_equal['households_left'].astype(int)>0]
<ipython-input-7-16e2f209a8e4>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  merge_income= merged_not_equal[merged_not_equal['households_right']==merged_not_equal['households_left']][merged_not_equal['yr_id']=='2022'][merged_not_equal['households_left'].astype(int)>0]


In [8]:
spatial_income = outlines_and_tracts.merge(merge_income, left_on='CT2010DT', right_on='tract',how='left')

# Finding Median Income

In [9]:
# Define a function to calculate the median of the income range
def calculate_median_income(income_range):
    if 'or more' in income_range:
        # Take the lower bound for 'or more'
        lower_bound = int(income_range.replace('$', '').replace('or more', '').replace(',', '').strip())
        return lower_bound
    elif 'Less than' in income_range:
        # Take the upper bound for 'Less than'
        upper_bound = int(income_range.replace('Less than $', '').replace(',', '').strip())
        return upper_bound
    else:
        # Split the range and calculate the median
        lower_bound, upper_bound = [int(s.replace('$', '').replace(',', '').strip()) for s in income_range.split('to')]
        return (lower_bound + upper_bound) // 2

# Median Income by Households

Separating Downtown Data by Tract and Most Recent Year

In [10]:
incomes_dt = income[income['tract']=='201.05']

In [11]:
median_income_2022_dt = incomes_dt[incomes_dt['yr_id'] == '2022']

In [12]:
median_income_2022_dt

,tract,yr_id,name,households
12410,201.05,2022,"Less than $15,000",63
12411,201.05,2022,"$15,000 to $29,999",140
12412,201.05,2022,"$30,000 to $44,999",105
12413,201.05,2022,"$45,000 to $59,999",162
12414,201.05,2022,"$60,000 to $74,999",155
12415,201.05,2022,"$75,000 to $99,999",175
12416,201.05,2022,"$100,000 to $124,999",150
12417,201.05,2022,"$125,000 to $149,999",125
12418,201.05,2022,"$150,000 to $199,999",245
12419,201.05,2022,"$200,000 or more",120


In [13]:
median_income_2022_dt['income'] = median_income_2022_dt['name'].apply(calculate_median_income)

<ipython-input-13-c20d6b777dc9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  median_income_2022_dt['income'] = median_income_2022_dt['name'].apply(calculate_median_income)


In [14]:
median_income_2022_dt

,tract,yr_id,name,households,income
12410,201.05,2022,"Less than $15,000",63,15000
12411,201.05,2022,"$15,000 to $29,999",140,22499
12412,201.05,2022,"$30,000 to $44,999",105,37499
12413,201.05,2022,"$45,000 to $59,999",162,52499
12414,201.05,2022,"$60,000 to $74,999",155,67499
12415,201.05,2022,"$75,000 to $99,999",175,87499
12416,201.05,2022,"$100,000 to $124,999",150,112499
12417,201.05,2022,"$125,000 to $149,999",125,137499
12418,201.05,2022,"$150,000 to $199,999",245,174999
12419,201.05,2022,"$200,000 or more",120,200000


In [15]:
# Assume median_income_2022_dt is your DataFrame with 'households' and 'income' columns
median_income_2022_dt['cumulative_frequency'] = median_income_2022_dt['households'].cumsum()
total_households = median_income_2022_dt['households'].sum()
median_position = total_households / 2
median_class_interval = median_income_2022_dt[median_income_2022_dt['cumulative_frequency'] >= median_position].iloc[0]

# Calculate the lower boundary of the median class interval
L = (median_class_interval['income'] - 
     (median_class_interval['income'] - 
      median_income_2022_dt[median_income_2022_dt['income'] < median_class_interval['income']]['income'].max()) / 2)

# If the median class is the first class, L would be 0 (or another reasonable assumption)
F = median_income_2022_dt[median_income_2022_dt['income'] < median_class_interval['income']]['households'].sum()
f = median_class_interval['households']
W = (median_income_2022_dt['income'].iloc[1] - median_income_2022_dt['income'].iloc[0])
median_income = L + ((median_position - F) / f) * W
median_income = round(median_income)

<ipython-input-15-6c6dd73f5388>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  median_income_2022_dt['cumulative_frequency'] = median_income_2022_dt['households'].cumsum()


# Median Household Income for Downtown Employment Center

In [16]:
median_income

81570

# Median Income by Household Region

In [17]:
incomes_region = pd.read_csv('data/2022_Estimates_Household_Income_by_2020_Census_Tract.csv')

In [18]:
incomes_region[incomes_region['yr_id'] == 2022]

,tract,yr_id,name,households
20,1.00,2022,"Less than $15,000",1
21,1.00,2022,"$15,000 to $29,999",9
22,1.00,2022,"$30,000 to $44,999",69
23,1.00,2022,"$45,000 to $59,999",44
24,1.00,2022,"$60,000 to $74,999",64
...,...,...,...,...
22075,99.02,2022,"$75,000 to $99,999",0
22076,99.02,2022,"$100,000 to $124,999",0
22077,99.02,2022,"$125,000 to $149,999",0
22078,99.02,2022,"$150,000 to $199,999",0


In [19]:
median_income_2022_region = incomes_region.groupby('name').sum().reset_index()
median_income_2022_region = median_income_2022_region[['name', 'households']]
median_income_2022_region

,name,households
0,"$100,000 to $124,999",360301
1,"$125,000 to $149,999",289765
2,"$15,000 to $29,999",276568
3,"$150,000 to $199,999",402192
4,"$200,000 or more",637730
5,"$30,000 to $44,999",282319
6,"$45,000 to $59,999",277735
7,"$60,000 to $74,999",284097
8,"$75,000 to $99,999",425536
9,"Less than $15,000",217205


In [20]:
median_income_2022_region['income'] = median_income_2022_region['name'].apply(calculate_median_income)

In [21]:
median_income_2022_region = median_income_2022_region.sort_values(by='income')

In [22]:
median_income_2022_region

,name,households,income
9,"Less than $15,000",217205,15000
2,"$15,000 to $29,999",276568,22499
5,"$30,000 to $44,999",282319,37499
6,"$45,000 to $59,999",277735,52499
7,"$60,000 to $74,999",284097,67499
8,"$75,000 to $99,999",425536,87499
0,"$100,000 to $124,999",360301,112499
1,"$125,000 to $149,999",289765,137499
3,"$150,000 to $199,999",402192,174999
4,"$200,000 or more",637730,200000


In [23]:
# Assume median_income_2022_dt is your DataFrame with 'households' and 'income' columns
median_income_2022_region['cumulative_frequency'] = median_income_2022_region['households'].cumsum()
total_households = median_income_2022_region['households'].sum()
median_position = total_households / 2
median_class_interval = median_income_2022_region[median_income_2022_region['cumulative_frequency'] >= median_position].iloc[0]

# Calculate the lower boundary of the median class interval
L = (median_class_interval['income'] - 
     (median_class_interval['income'] - 
      median_income_2022_region[median_income_2022_region['income'] < median_class_interval['income']]['income'].max()) / 2)

# If the median class is the first class, L would be 0 (or another reasonable assumption)
F = median_income_2022_region[median_income_2022_region['income'] < median_class_interval['income']]['households'].sum()
f = median_class_interval['households']
W = (median_income_2022_region['income'].iloc[1] - median_income_2022_region['income'].iloc[0])
median_income = L + ((median_position - F) / f) * W
median_income = round(median_income)

# Median Household Income for Region

In [24]:
median_income

84351

# Median Ages

In [25]:
#function to handle all cases including 'Under 5' and '85 and Older'
def calculate_median_age(age_range):
    # Handle the 'Under 5' case
    if age_range == 'Under 5':
        return 5  # Assuming median age between 0 and 5 is 2.5
    # Handle the '85 and Older' case
    elif age_range == '85 and Older':
        return 85  # Assuming 85 as the median age for '85 and Older'
    # Handle the 'and' in age range
    elif 'and' in age_range:
        # Take the average of the two numbers
        ages = age_range.replace('and', '').split()
        return sum(map(int, ages)) / 2
    # Handle the 'to' in age range
    else:
        ages = age_range.split(' to ')
        return (int(ages[0]) + int(ages[1])) / 2

In [26]:
import numpy as np

In [27]:
ages_region = pd.read_csv('data/2022_Estimates_Population_by_Age_by_2020_Census_Tract.csv')

# Median Age DT Employment Center

In [34]:
ages_dt = ages_region[ages_region['tract'] == 201.05]
ages_dt_2022 = ages_dt[ages_dt['yr_id'] == 2022]

In [35]:
ages_dt_2022.head()

,tract,yr_id,name,population
24820,201.05,2022,Under 5,207
24821,201.05,2022,5 to 9,229
24822,201.05,2022,10 to 14,204
24823,201.05,2022,15 to 17,92
24824,201.05,2022,18 and 19,36


In [36]:
# Apply the revised function to the 'name' column to create 'weighted age'
ages_dt_2022['weighted age'] = ages_dt_2022['name'].apply(calculate_median_age)
ages_dt_2022.head()

<ipython-input-36-04a01d71457e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages_dt_2022['weighted age'] = ages_dt_2022['name'].apply(calculate_median_age)


,tract,yr_id,name,population,weighted age
24820,201.05,2022,Under 5,207,5.0
24821,201.05,2022,5 to 9,229,7.0
24822,201.05,2022,10 to 14,204,12.0
24823,201.05,2022,15 to 17,92,16.0
24824,201.05,2022,18 and 19,36,18.5


instead of household use: population instead of income use weighted age

In [37]:
# Assume median_income_2022_dt is your DataFrame with 'households' and 'income' columns
ages_dt_2022['cumulative_frequency'] = ages_dt_2022['population'].cumsum()
total_population = ages_dt_2022['population'].sum()
median_position = total_population / 2
median_class_interval = ages_dt_2022[ages_dt_2022['cumulative_frequency'] >= median_position].iloc[0]

# Calculate the lower boundary of the median class interval
L = (median_class_interval['weighted age'] - 
     (median_class_interval['weighted age'] - 
      ages_dt_2022[ages_dt_2022['weighted age'] < median_class_interval['weighted age']]['weighted age'].max()) / 2)

# If the median class is the first class, L would be 0 (or another reasonable assumption)
F = ages_dt_2022[ages_dt_2022['weighted age'] < median_class_interval['weighted age']]['population'].sum()
f = median_class_interval['population']
W = (ages_dt_2022['weighted age'].iloc[1] - ages_dt_2022['weighted age'].iloc[0])
median_age_dt = L + ((median_position - F) / f) * W
median_age_dt = median_income

<ipython-input-37-0324401c58c8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages_dt_2022['cumulative_frequency'] = ages_dt_2022['population'].cumsum()


# Median Age of Downtown Employment Center

In [38]:
median_age_dt

84351

# Median Age for Region

In [39]:
ages_region_2022 = ages_region[ages_region['yr_id'] == 2022]

In [40]:
ages_region_2022['weighted age'] = ages_region_2022['name'].apply(calculate_median_age)
ages_region_2022 = ages_region_2022.groupby('weighted age').sum().reset_index()

<ipython-input-40-38ea213ca32f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ages_region_2022['weighted age'] = ages_region_2022['name'].apply(calculate_median_age)


In [41]:
ages_region_2022.head()

,weighted age,tract,yr_id,population
0,5.0,94131.46,1488192,187752
1,7.0,94131.46,1488192,212233
2,12.0,94131.46,1488192,211494
3,16.0,94131.46,1488192,125808
4,18.5,94131.46,1488192,88334


In [42]:
# Assume median_income_2022_dt is your DataFrame with 'households' and 'income' columns
ages_region_2022['cumulative_frequency'] = ages_region_2022['population'].cumsum()
total_population = ages_region_2022['population'].sum()
median_position = total_population / 2
median_class_interval = ages_region_2022[ages_region_2022['cumulative_frequency'] >= median_position].iloc[0]

# Calculate the lower boundary of the median class interval
L = (median_class_interval['weighted age'] - 
     (median_class_interval['weighted age'] - 
      ages_region_2022[ages_region_2022['weighted age'] < median_class_interval['weighted age']]['weighted age'].max()) / 2)

# If the median class is the first class, L would be 0 (or another reasonable assumption)
F = ages_region_2022[ages_region_2022['weighted age'] < median_class_interval['weighted age']]['population'].sum()
f = median_class_interval['population']
W = (ages_region_2022['weighted age'].iloc[1] - ages_region_2022['weighted age'].iloc[0])
median_age_region = L + ((median_position - F) / f) * W
median_age_region = median_income

# Median Age of Region

In [43]:
median_age_region

84351